In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re
from pathlib import Path

pd.set_option("display.max_columns", 500)

In [2]:
sampledf = pd.read_csv(Path("data", "sample.csv"), index_col=0)

/Users/rad/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/rad/anaconda3/lib/python3.7/site-packages/numpy/lib/arraysetops.py:522: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
df = sampledf.copy()

In [4]:
df.head(3)

,npi,nppes_provider_last_org_name,nppes_provider_first_name,nppes_provider_mi,nppes_credentials,nppes_provider_gender,nppes_entity_code,nppes_provider_street1,nppes_provider_street2,nppes_provider_city,nppes_provider_zip,nppes_provider_state,nppes_provider_country,provider_type,medicare_participation_indicator,place_of_service,hcpcs_code,hcpcs_description,hcpcs_drug_indicator,line_srvc_cnt,bene_unique_cnt,bene_day_srvc_cnt,average_medicare_allowed_amt,average_submitted_chrg_amt,average_medicare_payment_amt,year
8588549,1932288412,WARDEN,DAVID,R,MD,M,I,3100 CHANNING WAY,NaN,IDAHO FALLS,834047533,ID,US,Diagnostic Radiology,Y,O,76700,Ultrasound of abdomen,N,30.0,30,30,98.726000,268.733333,72.139000,2012
178269,1013992643,DETRANA,PHILIP,M,M.D.,M,I,2414 KOHLER MEMORIAL DR,NaN,SHEBOYGAN,530813129,WI,US,Pulmonary Disease,Y,O,36415,Insertion of needle into vein for collection o...,N,46.0,23,46,3.000000,3.000000,3.000000,2012
4234612,1467425520,ROSO,GEORGE,NaN,M.D.,M,I,255 S DOBSON RD,SUITE 1,CHANDLER,852246231,AZ,US,Family Practice,Y,O,90656,"Vaccine for influenza injection into muscle, p...",Y,11.0,11,11,12.389091,25.000000,11.261818,2012


In [5]:
df.shape

(1200000, 26)

In [6]:
# 64+489 = 553
((df.nppes_provider_country != 'US') | (df.medicare_participation_indicator == 'N')).sum()

553

In [7]:
df = df.loc[(df.nppes_provider_country == 'US') & (df.medicare_participation_indicator == 'Y')]
dropcols = ["nppes_provider_country", "medicare_participation_indicator"]
df = df.drop(dropcols, axis=1)

In [8]:
df.shape

(1199447, 24)

In [9]:
df.head(3)

,npi,nppes_provider_last_org_name,nppes_provider_first_name,nppes_provider_mi,nppes_credentials,nppes_provider_gender,nppes_entity_code,nppes_provider_street1,nppes_provider_street2,nppes_provider_city,nppes_provider_zip,nppes_provider_state,provider_type,place_of_service,hcpcs_code,hcpcs_description,hcpcs_drug_indicator,line_srvc_cnt,bene_unique_cnt,bene_day_srvc_cnt,average_medicare_allowed_amt,average_submitted_chrg_amt,average_medicare_payment_amt,year
8588549,1932288412,WARDEN,DAVID,R,MD,M,I,3100 CHANNING WAY,NaN,IDAHO FALLS,834047533,ID,Diagnostic Radiology,O,76700,Ultrasound of abdomen,N,30.0,30,30,98.726000,268.733333,72.139000,2012
178269,1013992643,DETRANA,PHILIP,M,M.D.,M,I,2414 KOHLER MEMORIAL DR,NaN,SHEBOYGAN,530813129,WI,Pulmonary Disease,O,36415,Insertion of needle into vein for collection o...,N,46.0,23,46,3.000000,3.000000,3.000000,2012
4234612,1467425520,ROSO,GEORGE,NaN,M.D.,M,I,255 S DOBSON RD,SUITE 1,CHANDLER,852246231,AZ,Family Practice,O,90656,"Vaccine for influenza injection into muscle, p...",Y,11.0,11,11,12.389091,25.000000,11.261818,2012


In [10]:
def summary(df_input):
    x = df.agg(["dtype","count","nunique",lambda x: x.isna().sum()]).rename(index={"<lambda>":"missing"}).T
    return x.sort_values(by="missing", ascending=False)

In [11]:
summary(df)

,dtype,count,nunique,missing
nppes_provider_street2,object,523224,44639,676223
nppes_provider_mi,object,869600,31,329847
nppes_credentials,object,1120640,7892,78807
nppes_provider_first_name,object,1147462,43255,51985
nppes_provider_gender,object,1147473,2,51974
nppes_provider_last_org_name,object,1199431,164629,16
npi,int64,1199447,551399,0
hcpcs_description,object,1199447,5034,0
average_medicare_payment_amt,float64,1199447,922746,0
average_submitted_chrg_amt,float64,1199447,355328,0


In [12]:
def imputeNone(df_input):
    df = df_input.copy()
    cols = ['nppes_provider_last_org_name', 'nppes_provider_first_name', 'nppes_provider_mi', 'nppes_credentials', 'nppes_provider_gender']
    for col in cols:
        df[col] = np.where(df['nppes_entity_code']=='O', "None", df[col])
    return(df)

In [13]:
df=imputeNone(df)

In [14]:
#Return summary with missingness updated with "None"
summary(df)

,dtype,count,nunique,missing
nppes_provider_street2,object,523224,44639,676223
nppes_provider_mi,object,921574,32,277873
nppes_credentials,object,1172614,7893,26833
nppes_provider_last_org_name,object,1199431,152139,16
nppes_provider_first_name,object,1199436,43256,11
npi,int64,1199447,551399,0
hcpcs_description,object,1199447,5034,0
average_medicare_payment_amt,float64,1199447,922746,0
average_submitted_chrg_amt,float64,1199447,355328,0
average_medicare_allowed_amt,float64,1199447,525597,0


In [15]:
num_features = df.dtypes[df.dtypes!=object].index
num_features.tolist()

['npi',
 'line_srvc_cnt',
 'bene_unique_cnt',
 'bene_day_srvc_cnt',
 'average_medicare_allowed_amt',
 'average_submitted_chrg_amt',
 'average_medicare_payment_amt',
 'year']

In [16]:
df_num = df[num_features]
df_num.sample(3)

,npi,line_srvc_cnt,bene_unique_cnt,bene_day_srvc_cnt,average_medicare_allowed_amt,average_submitted_chrg_amt,average_medicare_payment_amt,year
1247694,1134122765,176.0,23,176,25.980057,47.773864,17.727102,2015
1726253,1184698508,49.0,49,49,69.530000,123.000000,67.254490,2013
3809961,1407967482,199.0,28,199,117.620000,325.618090,92.515327,2013


In [17]:
df_num.describe()

,npi,line_srvc_cnt,bene_unique_cnt,bene_day_srvc_cnt,average_medicare_allowed_amt,average_submitted_chrg_amt,average_medicare_payment_amt,year
count,1.199447e+06,1.199447e+06,1.199447e+06,1.199447e+06,1.199447e+06,1.199447e+06,1.199447e+06,1.199447e+06
mean,1.499906e+09,2.362149e+02,8.931531e+01,1.456234e+02,9.897056e+01,3.240721e+02,7.591867e+01,2.014500e+03
std,2.879022e+08,3.552352e+03,1.067404e+03,2.067215e+03,2.365549e+02,9.089984e+02,1.845431e+02,1.707819e+00
min,1.003000e+09,9.500000e+00,1.100000e+01,1.100000e+01,1.666667e-04,1.666667e-04,0.000000e+00,2.012000e+03
25%,1.245647e+09,2.100000e+01,1.700000e+01,2.000000e+01,2.305000e+01,5.300000e+01,1.844000e+01,2.013000e+03
50%,1.508015e+09,4.400000e+01,3.300000e+01,4.100000e+01,6.259559e+01,1.340000e+02,4.554447e+01,2.015000e+03
75%,1.750304e+09,1.200000e+02,7.600000e+01,1.090000e+02,1.120400e+02,2.751753e+02,8.399000e+01,2.016000e+03
max,1.993000e+09,1.240597e+06,3.631420e+05,9.759500e+05,3.927928e+04,9.990000e+04,3.078999e+04,2.017000e+03
